**To-Do:**
- Backpropagation
- Scaling Class

In [15]:
from numpy.typing import NDArray
import numpy as np

In [16]:
class Layer:
    __supported_activations: list[str] = ["linear","relu","softmax"]

    def __init__(self, size: int, activation: str = "linear") -> None:
        if activation not in Layer.__supported_activations:
            raise ValueError(f"Parameter 'activation' not supported, please enter one of the following: {', '.join(Layer.__supported_activations)}")
        self.__size: int = size
        self.__activation: str = activation
        self.__parent: Layer = None
        self.__child: Layer = None

    def set_parent(self, parent: "Layer") -> None:
        self.__parent = parent

    def get_parent(self) -> "Layer":
        return self.__parent

    def set_child(self, child: "Layer") -> None:
        self.__child = child

    def get_child(self) -> "Layer":
        return self.__child

    def set_id(self, id: int) -> None:
        self.__id = id

    def get_id(self) -> int:
        return self.__id

    def get_size(self) -> int:
        return self.__size

    def get_activation(self) -> str:
        return self.__activation

    def set_matrices(self) -> None:
        if self.__parent:
            parent_size: int = self.__parent.get_size()
            self.__W: NDArray[np.float64] = np.random.normal(0, 0.01, size=(self.__size, parent_size))
            self.__B: NDArray[np.float64] = np.random.normal(0, 0.01, size=(self.__size, 1))
            self.__A: NDArray[np.float64] = np.zeros((self.__size, 1))
        else:
            self.__W: NDArray[np.float64] = None
            self.__B: NDArray[np.float64] = None
            self.__A: NDArray[np.float64] = None

    def get_value(self) -> NDArray[np.float64]:
        return self.__A

    def set_input(self, input: NDArray[np.float64]) -> None:
        self.__A  = input

    def g(self, input: NDArray[np.float64]) -> NDArray[np.float64]:
        if self.__activation == "relu":
            return np.maximum(input, 0)
        elif self.__activation == "softmax":
            nominator: NDArray[np.float64] = np.exp(input)
            denominator: int = np.sum(nominator)
            return nominator / denominator
        else:
            return input

    def forward(self) -> None:
        self.__A = self.g(np.matmul(self.__W, self.__parent.get_value()) + self.__B)


In [17]:
class NeuralNetwork:

    def build(self, layers: list[Layer]) -> None:
        self.__layers: list[Layer] = layers

        # Updating layers' relationships
        current: Layer = None
        id: int = 0
        for layer in layers:
            if current:
                current.set_child(layer)
                layer.set_parent(current)
            layer.set_id(id)
            current = layer

    def fit_data(self, input: NDArray[np.float64], target: NDArray[np.float64]) -> None:
        if input.size != self.__layers[0].get_size():
            raise ValueError(f"At the input layer, supported size is: {self.__layers[0].get_size()}, received: {input.size}")
        if target.size != self.__layers[-1].get_size():
            raise ValueError(f"At the target layer, supported size is: {self.__layers[-1].get_size()}, received: {target.size}")
        
        self.__target: NDArray[np.float64] = target
        
        # Creating weight matrix W, bias vector B, and value vector A for every layer
        # For a specifc layer containing K neurons, having a parent layer containing N neurons:
        # W size: (K,N) | B size: (K,1) | A size: (K,1)
        for layer in self.__layers:
            layer.set_matrices()
        self.__layers[0].set_input(input)

    def forward_propagation(self) -> None:
        for layer in self.__layers:
            if layer.get_parent():
                layer.forward()

    def get_results(self) -> NDArray[np.float64]:
        return self.__layers[-1].get_value()

In [18]:
test = NeuralNetwork()
test.build([
    Layer(3),
    Layer(10,"relu"),
    Layer(4,"relu"),
    Layer(20,"relu"),
    Layer(3,"softmax")
])
test.fit_data(np.array([[0.3],[0.2],[0.4]]),np.array([[0.3],[0.2],[0.4]]))
test.forward_propagation()
print(test.get_results())

[[0.33117408]
 [0.33419202]
 [0.33463391]]
